In [ ]:
import numpy as np
import pandas as pd
import torch.nn as nn
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import pairwise_distances
import random
from efficientnet_pytorch import EfficientNet
import pickle
import plotly.express as px

import matplotlib.pyplot as plt
from scipy import fftpack

import os
from pathlib import Path

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
file = open('./postion0', 'rb')
position = pickle.load(file)
file.close()

In [ ]:
file = open('./IM0', 'rb')
IM = pickle.load(file)
file.close()
IM = [np.transpose(i, (1,2,0)) for i in IM]

In [ ]:
file = open('./dat0', 'rb')
data = pickle.load(file)
file.close()

add color bar

low pass filtering

grayscale

depth position

In [ ]:
pos = pd.DataFrame(position).reset_index()

In [ ]:
from sklearn.cluster import SpectralClustering, KMeans
Clustering = KMeans(n_clusters=6, random_state=0, n_init="auto").fit(position[['anterior_posterior_ccf_coordinate','dorsal_ventral_ccf_coordinate','left_right_ccf_coordinate']])
pos['labels'] = Clustering.labels_

In [ ]:
fig = px.scatter_3d(pos, x = pos.anterior_posterior_ccf_coordinate, z =pos.dorsal_ventral_ccf_coordinate, y= pos.left_right_ccf_coordinate, hover_name = pos.index,color=pos.labels )
fig.update_traces(marker_size = 4)
fig.show()

In [ ]:
for i in range(len(IM)):
  plt.figure(figsize=(14, 4))  # Set the figure size as needed
  plt.subplot(1, 3, 1)  # 1 row, 2 columns, and position 1
  plt.imshow(IM[i][:,:,0],cmap= 'gray')
  plt.colorbar()
  plt.title('channel 1')

  plt.subplot(1, 3, 2)  # 1 row, 2 columns, and position 2
  plt.imshow(IM[i][:,:,1],cmap= 'gray')
  plt.colorbar()
  plt.title('channel 2')

  plt.subplot(1,3,3)
  plt.imshow(IM[i][:,:,2],cmap= 'gray')
  plt.colorbar()
  plt.title('channel 3')
  plt.suptitle( 'neuron {}'.format(i))
  j = i//5
  o = i%5
  plt.show()
  plt.close()

pairwise distance matrix


In [ ]:
data

In [ ]:
def diff_images(im_1,im_2):
  return sum(sum(sum((im_1-im_2)**2))**0.5)
diff = []
for i in range(259):
  dif = []
  for j in range(259):
    dif.append(diff_images(IM[i],IM[j]))
  diff.append(dif)

diff = np.array(diff)
np.fill_diagonal(diff, np.nan)

plt.figure(figsize=(10,10))
plt.imshow(diff, cmap = 'hot')
plt.colorbar()
plt.title('difference between the images')
plt.ylabel('neuron number')
plt.xlabel('neuron number')
plt.show()
plt.close()

In [ ]:
# goal is to only keep the strongest pixels to see where the attention of the neuron is pointed to
im = np.array(IM)
ma = im.max()
mi = im.min()
im_sup = im
#it is done per channel here
im_sup_0 =[im_sup[:,:,:,0]<np.quantile(im_sup[:,:,:,0],0.7)]
im_sup_1 = [im_sup[:,:,:,1]<np.quantile(im_sup[:,:,:,1],0.7)]
im_sup_2 = [im_sup[:,:,:,2]<np.quantile(im_sup[:,:,:,2],0.7)]
im_sup[:,:,:,0][im_sup_0[0]] = mi
im_sup[:,:,:,1][im_sup_1[0]] = mi
im_sup[:,:,:,2][im_sup_2[0]] = mi

In [ ]:
for i in range(207):

  plt.imshow((im_sup[i]-mi)/(ma-mi))
  plt.title(i)
  plt.show()

In [ ]:
from scipy import fftpack
import numpy as np
import imageio
from PIL import Image, ImageDraw


def low_pass(image1_np):
  """
  perform a low pass filter by transforming into fourrier domain multipling by a disk and back to spacial domain
  """
  #fft of image
  fft1 = fftpack.fftshift(fftpack.fft2(image1_np))

  #Create a low pass filter image
  x,y = image1_np.shape[0],image1_np.shape[1]
  #size of circle
  e_x,e_y=100,100
  #create a box
  bbox=((x/2)-(e_x/2),(y/2)-(e_y/2),(x/2)+(e_x/2),(y/2)+(e_y/2))

  low_pass=Image.new("L",(image1_np.shape[0],image1_np.shape[1]),color=0)

  draw1=ImageDraw.Draw(low_pass)
  draw1.ellipse(bbox, fill=1)

  low_pass_np=np.array(low_pass)

  #multiply both the images
  filtered=np.multiply(fft1,low_pass_np)

  #inverse fft
  ifft2 = np.real(fftpack.ifft2(fftpack.ifftshift(filtered)))
  ifft2 = np.maximum(0, np.minimum(ifft2, 255))
  return ifft2
  #plt.imshow(ifft2,cmap = 'gray')

def rgb2gray(rgb):
    #turns the rgb channel to black and white according to wikipedia def
    r, g, b = rgb[:,:,0], rgb[:,:,1], rgb[:,:,2]
    gray = 0.2989 * r + 0.5870 * g + 0.1140 * b
    return gray

def meanrgb(rgb):
    #turns the rgb channel to black and white according to wikipedia def
    r, g, b = rgb[:,:,0], rgb[:,:,1], rgb[:,:,2]
    gray =  0.33* r + 0.33* g + 0.33 * b
    return gray

In [ ]:

gray_im_rgb = []
for i in IM:
  #we transform into grayscale using the mean of the channels
  gray_im_rgb.append(rgb2gray(i))
gray_im_rgb = np.array(gray_im_rgb)
ma_rgb = gray_im_rgb.max()
mi_rgb = gray_im_rgb.min()
gray_im = []
for i in IM:
  #we transform into grayscale using the rgb decomposition
  gray_im.append(meanrgb(i))
gray_im = np.array(gray_im)
ma = gray_im.max()
mi = gray_im.min()
#gray_im[gray_im<np.quantile(gray_im,0.7)] = mi

for i in range(207):
  plt.figure(figsize=(10, 4))  # Set the figure size as needed
  plt.subplot(1, 2, 1)
  plt.imshow((gray_im[i]-mi)/(ma-mi), cmap = 'gray')
  plt.colorbar()
  plt.title('MEI for neuron {} with mean over channels'.format(i))
  plt.subplot(1, 2, 2)
  plt.imshow((gray_im_rgb[i]-mi_rgb)/(ma_rgb-mi_rgb), cmap = 'gray')
  plt.colorbar()
  plt.title('MEI for neuron {} with rgb decomp over channels'.format(i))
  plt.show()
  plt.close()


check gaussian values
plot neuronal answer rate per image and MEI and also lowpass highpass

In [ ]:
from scipy import ndimage
low_p_rgb = []
high_p_rgb = []
low_p = []
high_p = []
for i in range(len(IM)):
  lowpass = ndimage.gaussian_filter(gray_im_rgb[i], 5)
  gauss_highpass = gray_im[i] - lowpass
  low_p_rgb.append(lowpass)
  high_p_rgb.append(gauss_highpass)
  lowpass = ndimage.gaussian_filter(gray_im[i], 10)
  gauss_highpass = gray_im[i] - lowpass
  low_p.append(lowpass)
  high_p.append(gauss_highpass)
  plt.figure(figsize=(14, 4))  # Set the figure size as needed
  plt.subplot(1, 2, 1)
  plt.imshow(gauss_highpass, cmap = 'gray')
  plt.colorbar()
  plt.title('highpass {}'.format(i))
  plt.subplot(1, 2, 2)
  plt.imshow(lowpass, cmap = 'gray')
  plt.colorbar()
  plt.title('lowpass {}'.format(i))
  plt.show()
  plt.close()

In [ ]:
low_p = np.array(low_p)
mi = low_p.min()
low_p[low_p<np.quantile(low_p,0.7)] = mi
for i in low_p:
  plt.imshow(i, cmap= 'gray')
  plt.show()
  plt.close()

In [ ]:
def diff_images_gray(im_1,im_2):
  return sum(sum((im_1-im_2)**2))**0.5
diff_l = []
diff_h = []
for i in range(259):
  dif_r = []
  dif = []
  for j in range(259):
    dif_r.append(diff_images_gray(low_p[i],low_p[j]))
    dif.append(diff_images_gray(high_p[i],high_p[j]))
  diff_l.append(dif_r)
  diff_h.append(dif)
diff_l = np.array(diff_l) 
diff_h = np.array(diff_h)
np.fill_diagonal(diff_h, np.nan)
np.fill_diagonal(diff_l, np.nan)


plt.rc('font', size=16)

plt.figure(figsize=(20, 20))
plt.subplot(2, 2, 1)
plt.imshow(diff, cmap = 'hot')
plt.colorbar()
plt.title('difference between the images')
plt.ylabel('neuron number')
plt.xlabel('neuron number')

plt.subplot(2, 2, 4)
plt.imshow(diff_h, cmap = 'hot')
plt.colorbar()
plt.title('difference between high frequencies')
plt.ylabel('neuron number')
plt.xlabel('neuron number')

plt.subplot(2, 2, 3)
plt.imshow(diff_l, cmap = 'hot')
plt.colorbar()
plt.title('difference between low frequencies')
plt.ylabel('neuron number')
plt.xlabel('neuron number')

temp = pairwise_distances(np.array(data.drop(columns='count')).T)
plt.subplot(2, 2, 2)
plt.imshow(temp, cmap = 'hot')
plt.colorbar()
plt.title('pairwise distance between the neuron allensdk data')
plt.tight_layout()

In [ ]:
def diff_images_gray(im_1,im_2):
  return sum(sum((im_1-im_2)**2))**0.5
diff_rgb = []
diff = []
for i in range(206):
  dif_r = []
  dif = []
  for j in range(206):
    if i!=j:
      dif_r.append(diff_images_gray(gray_im_rgb[i],gray_im_rgb[j]))
      dif.append(diff_images_gray(gray_im[i],gray_im[j]))
    else:
      dif.append(np.nan)
      dif_r.append(np.nan)
  diff_rgb.append(dif_r)
  diff.append(dif)
diff_ma = np.array(diff).max()
diff_mi = np.array(diff).min()
diff_rgb_ma = np.array(diff_rgb).max()
diff_rgb_mi = np.array(diff_rgb).min()

plt.figure(figsize=(30, 10))

plt.subplot(1, 3, 1)
plt.imshow(diff, cmap = 'hot')
plt.colorbar()
plt.title('difference between the grascale mean channels')

plt.subplot(1, 3, 2)
plt.imshow(diff_rgb, cmap = 'hot')
plt.colorbar()
plt.title('difference between the grascale rgb transformed channels')

temp = pairwise_distances(np.array(data.drop(columns='count')).T)
plt.subplot(1, 3, 3)
plt.imshow(temp, cmap = 'hot')
plt.colorbar()
plt.title('difference between the neuron allensdk data data')

In [ ]:
from sklearn.cluster import SpectralClustering, KMeans, DBSCAN, OPTICS
Clustering =  SpectralClustering(n_clusters=6,
         assign_labels='discretize',
         random_state=0).fit(np.array(low_p).reshape((259,50176)))
pos['labels'] = Clustering.labels_

In [ ]:
fig = px.scatter_3d(pos, x = pos.anterior_posterior_ccf_coordinate, z =pos.dorsal_ventral_ccf_coordinate, y= pos.left_right_ccf_coordinate, hover_name = pos.index,color=pos.labels )
fig.update_traces(marker_size = 4)
fig.show()

In [ ]:
gray_im_sup = np.expand_dims(gray_im, axis=-1)
#only keep the 30 top quantile of pixels to display
#gray_im_sup[gray_im_sup[:,:,:,0]<np.quantile(gray_im_sup[:,:,:,0],0.7)]= 0
#gray_im_sup[gray_im_sup[:,:,:,0]>np.quantile(gray_im_sup[:,:,:,0],0.7)]= 1
x,y,z = gray_im_sup.reshape(259,224,224).nonzero()
temp=pd.DataFrame(np.array(gray_im_sup.reshape(259,224,224).nonzero())).T

In [ ]:
temp = np.array(gray_im).reshape(259,224,224)
for i in range(224):
  plt.figure(figsize=(8, 4))  # Set the figure size as needed
  plt.subplot(1, 2, 1)
  plt.imshow(temp[:,i],cmap = 'gray')
  plt.ylabel('neuron number')
  plt.xlabel('horizontal axis of MEI')

  plt.subplot(1, 2, 2)
  plt.imshow(temp[:,:,i],cmap = 'gray')
  plt.suptitle('position {}'.format(i))
  plt.ylabel('neuron number')
  plt.xlabel('vertical axis of MEI')
  plt.show()
  plt.close()


### Fourrier analysis

In [ ]:
def f(im):
  return np.log(np.abs(fftpack.fftshift(torch.fft.fft2(torch.tensor(im)).detach().float())+0.00000000000000001))


def Magnitude_Spectrum_Statistics(image,i):
  fourier_transform = fftpack.fftn(image[:,:,0])

  # Shift zero frequency components to the center
  fourier_transform_shifted = fftpack.fftshift(fourier_transform)

  # Calculate magnitude spectrum
  magnitude_spectrum = np.abs(fourier_transform_shifted)

  # Compute Magnitude Spectrum Statistics
  mean_magnitude = np.mean(magnitude_spectrum)
  std_magnitude = np.std(magnitude_spectrum)
  max_magnitude = np.max(magnitude_spectrum)
  entropy_magnitude = -np.sum((magnitude_spectrum / np.sum(magnitude_spectrum)) * np.log2(magnitude_spectrum / np.sum(magnitude_spectrum)))
  plt.figure(figsize=(12, 14))
  plt.subplot(3, 3, 1)
  plt.imshow(image[:,:,0], cmap='gray')
  plt.title('Original Image')

  plt.subplot(3, 3, 2)
  plt.imshow(np.log1p(magnitude_spectrum), cmap='gray')
  plt.title('Magnitude Spectrum (log-scaled)')

  plt.subplot(3, 3, 3)
  plt.bar(['Mean', 'Std', 'Max', 'Entropy'], [mean_magnitude, std_magnitude, max_magnitude, entropy_magnitude],log = True)
  plt.title('Magnitude Spectrum Statistics')


  fourier_transform = fftpack.fftn(image[:,:,1])

  # Shift zero frequency components to the center
  fourier_transform_shifted = fftpack.fftshift(fourier_transform)

  # Calculate magnitude spectrum
  magnitude_spectrum = np.abs(fourier_transform_shifted)

  # Compute Magnitude Spectrum Statistics
  mean_magnitude = np.mean(magnitude_spectrum)
  std_magnitude = np.std(magnitude_spectrum)
  max_magnitude = np.max(magnitude_spectrum)
  entropy_magnitude = -np.sum((magnitude_spectrum / np.sum(magnitude_spectrum)) * np.log2(magnitude_spectrum / np.sum(magnitude_spectrum)))
  plt.subplot(3, 3, 4)
  plt.imshow(image[:,:,1], cmap='gray')
  plt.title('Original Image')

  plt.subplot(3, 3, 5)
  plt.imshow(np.log1p(magnitude_spectrum), cmap='gray')
  plt.title('Magnitude Spectrum (log-scaled)')

  plt.subplot(3, 3, 6)
  plt.bar(['Mean', 'Std', 'Max', 'Entropy'], [mean_magnitude, std_magnitude, max_magnitude, entropy_magnitude],log = True)
  plt.title('Magnitude Spectrum Statistics')

  fourier_transform = fftpack.fftn(image[:,:,2])

  # Shift zero frequency components to the center
  fourier_transform_shifted = fftpack.fftshift(fourier_transform)

  # Calculate magnitude spectrum
  magnitude_spectrum = np.abs(fourier_transform_shifted)

  # Compute Magnitude Spectrum Statistics
  mean_magnitude = np.mean(magnitude_spectrum)
  std_magnitude = np.std(magnitude_spectrum)
  max_magnitude = np.max(magnitude_spectrum)
  entropy_magnitude = -np.sum((magnitude_spectrum / np.sum(magnitude_spectrum)) * np.log2(magnitude_spectrum / np.sum(magnitude_spectrum)))
  plt.subplot(3, 3, 7)
  plt.imshow(image[:,:,2], cmap='gray')
  plt.title('Original Image')

  plt.subplot(3, 3, 8)
  plt.imshow(np.log1p(magnitude_spectrum), cmap='gray')
  plt.title('Magnitude Spectrum (log-scaled)')

  plt.subplot(3, 3, 9)
  plt.bar(['Mean', 'Std', 'Max', 'Entropy'], [mean_magnitude, std_magnitude, max_magnitude, entropy_magnitude],log = True)
  plt.title('Magnitude Spectrum Statistics')


  plt.suptitle('image{}'.format(i))
  plt.savefig('drive/MyDrive/data/fourriers/image{}.png'.format(i))
  plt.show()


In [ ]:
for i in range(259):
  plt.imshow(f(low_p[i]), cmap = 'gray')
  plt.colorbar()
  plt.title(i)
  plt.show()

In [ ]:
def get_stats(image):
  fourier_transform = fftpack.fftn(image)
  # Shift zero frequency components to the center
  fourier_transform_shifted = fftpack.fftshift(fourier_transform)


  # Calculate magnitude spectrum
  magnitude_spectrum = np.abs(fourier_transform_shifted)
  # Compute Magnitude Spectrum Statistics
  mean_magnitude = np.median(magnitude_spectrum)
  std_magnitude = np.std(magnitude_spectrum)
  max_magnitude = np.max(magnitude_spectrum)
  entropy_magnitude = -np.sum((magnitude_spectrum / np.sum(magnitude_spectrum)) * np.log2(magnitude_spectrum / np.sum(magnitude_spectrum)))
  return mean_magnitude,std_magnitude,max_magnitude,entropy_magnitude

def mean_image_stat(im):
  mean_magnitude,std_magnitude,max_magnitude,entropy_magnitude = 0,0,0,0
  for i in range(3):
    magnitude,std,max,entropy = get_stats(im[:,:,i])
    mean_magnitude +=magnitude/3
    std_magnitude += std/3
    max_magnitude += max/3
    entropy_magnitude += entropy/3
  return mean_magnitude,std_magnitude,max_magnitude,entropy_magnitude



In [ ]:

image_stats = []
for i in range(len(IM)):
  image_stats.append(get_stats(meanrgb(IM[i])))
image_stats = np.array(image_stats)
mesures =['median','std', 'max']

for i in range(3):
  plt.figure(figsize=(12, 14))
  plt.subplot(4, 1, 1)
  plt.bar(range(259),image_stats[:,i],log = True)
  plt.title('{} Magnitude Spectrum'.format(mesures[i]))
  plt.xlabel('Neurons')
  plt.ylabel('Magnitude')

In [ ]:
l = []
div = 4
size = int(224/div)
l.append(np.zeros((224,224)))
for i in range(div):
  temp = np.pad(np.ones((i*size+size,i*size+size)), ((int((224-(i*size+size))/2), int((224-(i*size+size))/2)), (int((224-(i*size+size))/2), int((224-(i*size+size))/2))), 'constant', constant_values=(0, 0))
  l.append(temp)
for i in range(div+1):
  for j in range(div+1):
    if i>j:
      l[i] = l[i] - l[j]
l= l[1:]


In [ ]:
def get_stats_freq(image, freq):
  fourier_transform = fftpack.fftn(image)
  # Shift zero frequency components to the center
  fourier_transform_shifted = fftpack.fftshift(fourier_transform)
  fourier_transform_shifted = fourier_transform_shifted*freq
  # Calculate magnitude spectrum
  index = np.unravel_index(fourier_transform_shifted.argmax(), fourier_transform_shifted.shape)
  magnitude_spectrum = np.abs(fourier_transform_shifted)
  # Compute Magnitude Spectrum Statistics
  mean_magnitude = np.mean(magnitude_spectrum)
  std_magnitude = np.std(magnitude_spectrum)
  max_magnitude = np.max(magnitude_spectrum)
  return mean_magnitude,std_magnitude,max_magnitude
mesures =['median','std', 'max']
frequency =['low','medium_low', 'medium_high', 'high']
for j in range(div):
  image_stats = []
  for i in range(len(IM)):
    image_stats.append(get_stats_freq(rgb2gray(IM[i]), l[j]))
  image_stats = np.array(image_stats)
  for i in range(2):
    plt.figure(figsize=(12, 14))
    plt.subplot(4, 1, i+1)
    plt.bar(range(259),image_stats[:,i],log = True)
    plt.title('{} Magnitude Spectrum {} frequencies'.format(mesures[i],frequency[j]))
    plt.xlabel('neurons')
    plt.ylabel('magnitude')


In [ ]:
quarter = np.vstack((np.hstack((np.zeros((112,112)),np.zeros((112,112)))), np.hstack((np.ones((112,112)),np.zeros((112,112))))))

In [ ]:
def get_argmax_freq(image, freq):
  fourier_transform = fftpack.fftn(image)
  # Shift zero frequency components to the center
  fourier_transform_shifted = fftpack.fftshift(fourier_transform)
  fourier_transform_shifted = fourier_transform_shifted*freq
  # Calculate magnitude spectrum
  index = np.unravel_index(fourier_transform_shifted.argmax(), fourier_transform_shifted.shape)
  max_value = fourier_transform_shifted.max()
  return index[0], index[1], max_value

image_ind = []
for j in range(div):
  image_stats = []
  for i in range(len(IM)):
    image_stats.append(get_argmax_freq(rgb2gray(IM[i]), l[j]))
  image_ind.append(image_stats)

In [ ]:
df = pd.DataFrame(image_ind).T.reset_index()
df = df.melt(id_vars=['index'], value_vars=range(div))

df['x'], df['y'], df['maximum'] = zip(*df['value'])
df.maximum = np.clip(df.maximum.astype(int),0,400)
fig = px.scatter_3d(df, x= 'x', y = 'y', z = 'index', color= 'maximum', symbol = 'variable')
fig.update_traces(marker_size = 4)
fig.show()


In [ ]:
from sklearn.cluster import SpectralClustering, KMeans, DBSCAN, OPTICS
Clustering =  KMeans(n_clusters=6,
         random_state=0).fit(image_stats)
pos['labels'] = Clustering.labels_

In [ ]:
fig = px.scatter_3d(pos, x = pos.anterior_posterior_ccf_coordinate, z =pos.dorsal_ventral_ccf_coordinate, y= pos.left_right_ccf_coordinate, hover_name = pos.index,color=pos.labels )
fig.update_traces(marker_size = 4)
fig.show()

In [ ]:
for i in range(len(IM)):
  for j in range(3):
    # Compute 2D Fourier transform
    fourier_transform = fftpack.fft2(IM[i][:,:,j])

    # Shift zero frequency components to the center
    fourier_transform_shifted = fftpack.fftshift(fourier_transform)

    # Calculate power spectrum (squared magnitude)
    power_spectrum = np.abs(fourier_transform_shifted)**2

    # Display the original IM[i][:,] and its power spectrum
    plt.figure(figsize=(10, 5))

    plt.subplot(1, 2, 1)
    plt.imshow(IM[i][:,:,j], cmap='gray')
    plt.title('Original Image')

    plt.subplot(1, 2, 2)
    plt.imshow(np.log1p(power_spectrum), cmap='gray')
    plt.title('Power Spectrum (log-scaled)')

    plt.show()

In [ ]:
x = pd.DataFrame(low_p[0][:,:,0])
x = pd.melt(x.reset_index(), id_vars ='index', value_vars = x.columns)

In [ ]:
plt.imshow(low_p[30])

In [ ]:
import plotly.graph_objects as go
x = low_p[80]
fig = go.Figure(data=[go.Surface(z=pd.DataFrame(x), x=np.linspace(0,224,224), y=np.linspace(0,224,224))])
fig.show()

In [ ]:

#for i in IM:
fig = px.scatter_3d(x, x ='index', y= 'variable', z = 'value' )
fig.update_traces(marker_size = 1)
fig.show()

In [ ]:

for i in range(len(IM)):
    # Set the figure size as needed
    # 1 row, 2 columns, and position 1

  Magnitude_Spectrum_Statistics(IM[i],i)


### Wavelet transform

In [ ]:
xfm = DTCWTForward(J=3, biort='near_sym_b', qshift='qshift_b')
Yl, Yh = xfm(torch.movedim(torch.tensor(IM).float(),-1,1))
for i in range(len(IM)):
  for j in range(Yh[0].shape[2]):
    plt.figure(figsize=(10, 4))  # Set the figure size as needed
    plt.subplot(1, 3, 1)  # 1 row, 2 columns, and position 1
    plt.imshow(f(Yh[0][:,:,j,:,:,0][i][0,:,:]))#cho

    plt.subplot(1, 3, 2)  # 1 row, 2 columns, and position 2
    plt.imshow(f(Yh[0][:,:,j,:,:,0][i][1,:,:]))

    plt.subplot(1,3,3)
    plt.imshow(f(Yh[0][:,:,j,:,:,0][i][2,:,:]))
    plt.suptitle('image{} angle{}'.format(i,j))
    plt.show()